In [1]:
import numpy as np
import re
from sklearn.datasets import load_files
import nltk

# Pass the path to the folder containing the 'neg' & 'pos' subfolders.
# Check the link to get a better understanding of how the magical function 'load_files' functions
nucle = load_files(r"/Users/highsierra/Tech-Skills/Labortory/ML-Fundamentals/esmtxt_sentoken_conll14")
X, y = nucle.data, nucle.target

In [2]:
documents = []

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r"[^a-zA-Z/'/]", ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b' and ending with 'n'
    document = re.sub(r"^b\'", '', document)
    document = re.sub(r"^b", '', document)
    document = re.sub(r"n\'", '', document)

    documents.append(document)

In [3]:
import spacy

post = []
full = []

nlp = spacy.load("en_core_web_sm")
for text in documents:
    doc = nlp(text)
    for token in doc:
        post.append(token.pos_)
    post = []
    full.append(post)

In [4]:
for i in range(200):
    while len(full[i]) < 55:
        full[i].append('SPACE')

In [5]:
a = np.array(full)

In [6]:
print(a.shape)

(200, 55)


In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(), [0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 ,
14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 , 32 , 33 , 34 , 35 , 36 , 37 , 38 ,
39 , 40 , 41 , 42 , 43 , 44 , 45 , 46 , 47 , 48 , 49 , 50 , 51 , 52 , 53 , 54])], remainder='passthrough')

a = transformer.fit_transform(a)
print(a.toarray().shape)
a = a.toarray()
print(a)

(200, 530)
[[0. 0. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 ...
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 0. 1.]]


In [8]:
from zeugma.embeddings import EmbeddingTransformer

glove = EmbeddingTransformer('glove') 
X = glove.transform(documents)

Using TensorFlow backend.


In [9]:
print("concatenate column wise")
concat = np.concatenate((X,a),axis=1)
print(concat)

concatenate column wise
[[-0.00557672  0.12866175 -0.28674632 ...  1.          0.
   1.        ]
 [-0.0459268   0.33395836 -0.33841449 ...  1.          0.
   1.        ]
 [ 0.28554589  0.34604859 -0.27926505 ...  1.          0.
   1.        ]
 ...
 [ 0.1303066   0.10723669 -0.47099614 ...  1.          0.
   1.        ]
 [ 0.42725652  0.42168248 -0.18633699 ...  1.          0.
   1.        ]
 [ 0.05842838  0.136352   -0.11073975 ...  1.          0.
   1.        ]]


In [10]:
print(concat.shape)

(200, 555)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(concat, y, test_size=0.2, random_state=0)

In [12]:
X_train = X_train.reshape(-1, 1, 555)
X_test  = X_test.reshape(-1, 1, 555)
y_train = y_train.reshape(-1, 1, )
y_test = y_test.reshape(-1, 1, )

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [14]:
model = Sequential()
model.add(LSTM(200, input_shape=(1, 555), dropout=0.2, recurrent_dropout=0.2, return_sequences=True)) 
model.add(LSTM(100, input_shape=(1, 555), dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
model.fit(X_train, y_train, epochs=25, batch_size=64)


Epoch 1/25
160/160 [==============================] - 5s 29ms/step - loss: 0.6941 - accuracy: 0.5188
Epoch 2/25
160/160 [==============================] - 0s 871us/step - loss: 0.6866 - accuracy: 0.5250
Epoch 3/25
160/160 [==============================] - 0s 1ms/step - loss: 0.6823 - accuracy: 0.6625
Epoch 4/25
160/160 [==============================] - 0s 906us/step - loss: 0.6758 - accuracy: 0.7563
Epoch 5/25
160/160 [==============================] - 0s 871us/step - loss: 0.6647 - accuracy: 0.7250
Epoch 6/25
160/160 [==============================] - 0s 929us/step - loss: 0.6513 - accuracy: 0.7188
Epoch 7/25
160/160 [==============================] - 0s 845us/step - loss: 0.6289 - accuracy: 0.7875
Epoch 8/25
160/160 [==============================] - 0s 930us/step - loss: 0.6053 - accuracy: 0.8125
Epoch 9/25
160/160 [==============================] - 0s 836us/step - loss: 0.5658 - accuracy: 0.7812
Epoch 10/25
160/160 [==============================] - 0s 986us/step - loss: 0.5318 

In [16]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 40.00%
